<a href="https://colab.research.google.com/github/KrzRac/UGP/blob/main/dekoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import json

In [4]:
from google.colab import drive
import os

drive.mount('/content/drive')

drive_results_dir = "/content/drive/MyDrive/gpt_2_results"
os.makedirs(drive_results_dir, exist_ok=True)

Mounted at /content/drive


In [5]:
dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [7]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token by default
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [9]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})


In [10]:
train_dataset = tokenized_datasets["train"]
valid_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

In [11]:
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
frozen_layer_count = 4  # Number of decoder layers to freeze
for name, param in model.named_parameters():
    if "transformer.h." in name and int(name.split(".")[2]) < frozen_layer_count:
        param.requires_grad = False

In [13]:
print("GPT2ForSequenceClassification:")
print(model)
frozen_layers = [name for name, param in model.named_parameters() if not param.requires_grad]
print("Frozen layers:", frozen_layers)

GPT2ForSequenceClassification:
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)
Frozen lay

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc}

In [17]:
training_args = TrainingArguments(
    output_dir=drive_results_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir=drive_results_dir,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-13b66fff041f>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
print("Training GPT2ForSequenceClassification...")
trainer.train()

Training GPT2ForSequenceClassification...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.431900,0.405171,0.825516
2,0.318300,0.315065,0.863039
3,0.305800,0.313156,0.868668
4,0.233800,0.314923,0.871482
5,0.355600,0.341125,0.860225


TrainOutput(global_step=2670, training_loss=0.33576811596695405, metrics={'train_runtime': 708.4582, 'train_samples_per_second': 60.201, 'train_steps_per_second': 3.769, 'total_flos': 1663467169480704.0, 'train_loss': 0.33576811596695405, 'epoch': 5.0})

In [24]:
history = trainer.state.log_history

history_file = os.path.join(drive_results_dir, "training_history.json")
with open(history_file, "w") as f:
    json.dump(history, f, indent=4)
print(f"Training history has been saved to {history_file}.")

Training history has been saved to /content/drive/MyDrive/gpt_2_results/training_history.json.


In [21]:
eval_results = trainer.evaluate(test_dataset)

eval_results_file = os.path.join(drive_results_dir, "eval_results.txt")
with open(eval_results_file, "w") as f:
    f.write(str(eval_results))

print(f"Evaluation results saved at: {eval_results_file}")

print("Evaluation results:", eval_results)

Evaluation results saved at: /content/drive/MyDrive/gpt_2_results/eval_results.txt
Evaluation results: {'eval_loss': 0.3470243811607361, 'eval_accuracy': 0.8611632270168855, 'eval_runtime': 4.0292, 'eval_samples_per_second': 264.567, 'eval_steps_per_second': 16.628, 'epoch': 5.0}


In [25]:
print("Generating detailed evaluation report...")
logits, labels = trainer.predict(test_dataset)[:2]
predictions = np.argmax(logits, axis=-1)
report = classification_report(labels, predictions, target_names=["negative", "positive"], digits=4)

classification_report_file = os.path.join(drive_results_dir, "classification_report.txt")
with open(classification_report_file, "w") as f:
    f.write(report)

print(f"Classification report saved at: {classification_report_file}")

print("Classification Report:\n", report)

Generating detailed evaluation report...


Classification report saved at: /content/drive/MyDrive/gpt_2_results/classification_report.txt
Classification Report:
               precision    recall  f1-score   support

    negative     0.8494    0.8780    0.8635       533
    positive     0.8738    0.8443    0.8588       533

    accuracy                         0.8612      1066
   macro avg     0.8616    0.8612    0.8611      1066
weighted avg     0.8616    0.8612    0.8611      1066



In [26]:
model_save_path = os.path.join(drive_results_dir, "gpt2_finetuned")
tokenizer_save_path = os.path.join(drive_results_dir, "gpt2_finetuned_tokenizer")

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved at: {model_save_path}")
print(f"Tokenizer saved at: {tokenizer_save_path}")

Model saved at: /content/drive/MyDrive/gpt_2_results/gpt2_finetuned
Tokenizer saved at: /content/drive/MyDrive/gpt_2_results/gpt2_finetuned_tokenizer


In [27]:
examples = test_dataset['text'][:3]

tokens = tokenizer(examples, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Ensure the model and data are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokens = {key: val.to(device) for key, val in tokens.items()}

# Get predictions
outputs = model(**tokens)
predictions = torch.argmax(outputs.logits, dim=-1)

print("Predictions:")
for i, example in enumerate(examples):
    print(f"Review: {example}")
    print(f"Predicted sentiment: {'positive' if predictions[i] == 1 else 'negative'}")


Predictions:
Review: lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .
Predicted sentiment: positive
Review: consistently clever and suspenseful .
Predicted sentiment: positive
Review: it's like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .
Predicted sentiment: negative
